***
## Как написать пагинатор

Чтобы на каждой странице выводить, например, по десять записей, — надо:

* в HTML-шаблон добавить элемент управления для перехода по страницам: набор ссылок, каждая из которых отправляет GET-запрос с параметром `page`, значение которого — номер запрошенной страницы, например `/?page=2`;

* доработать view-функцию:

    * при запросе к БД отсортировать объекты (например по id). Иначе порядок объектов на одной и той же странице может отличаться просто из-за того, что БД вернёт объекты в другом порядке.

    * добавить проверку, есть ли `page` в GET-параметрах:

        * если в параметрах нет `page` или значение невозможно привести к числу — отдавать первую страницу с записями с первой по десятую;

        * если `page` присутствует, то с помощью методов ORM `offset()` и `limit()` получить нужный диапазон записей. Для `/?page=2` нужно будет получить записи с одиннадцатой по двадцатую включительно.

В Django есть готовое решение, позволяющее реализовать пагинацию с минимальными трудозатратами. Встроенный класс `Paginator` раскладывает списки объектов по отдельным страницам и предоставляет дополнительные возможности для управления такими страницами. 

Подключим пагинацию к страницам приложения **birthday**; прежде чем погрузиться в код — посмотрим, как работает `Paginator`.

***
## Класс Paginator

Объект класса `Paginator` получает на вход список объектов, разбивает его на отдельные страницы, позволяет обратиться к этим страницам по отдельности или получить полный список страниц. 

Чтобы разбить список объектов на страницы, разработчику нужно:

* получить из БД полный список объектов, который нужно разбить на страницы,

* создать объект класса `Paginator`,

* передать в объект класса `Paginator` список объектов и число — количество элементов, которое нужно выводить на страницу.

Всё остальное модуль сделает сам, остаётся только обращаться к свойствам и методам созданного объекта.

Импортируем класс `Paginator`, создадим список объектов (классу `Paginator` нет разницы, откуда этот список взялся — получен ли он из БД или создан вручную), создадим объект класса `Paginator` и разделим список на части — по две записи на одной «странице».

```py
>>> from django.core.paginator import Paginator
# Создаём тестовый список объектов.
>>> items = ['Антон Чехов', 'Владимир Набоков', 'Лев Толстой', 'Марина Цветаева']

# Cоздаём объект Paginator(object_list, per_page),
# он принимает на вход список объектов 
# и число объектов, которое должно отображаться на одной странице.
# Передаём в Paginator список items и указываем "2 объекта на страницу".
>>> paginator = Paginator(items, 2)

# Свойство count показывает, сколько всего объектов в полученном списке.
>>> paginator.count
4

# Свойство num_pages показывает, сколько страниц получится из списка.
>>> paginator.num_pages
```

***
## Добавление пагинации к проекту

Теории достаточно; приложим её к практике — добавим постраничное разбиение списка, который выводится на страницу *birthday/list/*.

* Во view-функции `birthday_list()` создадим объект пагинатора и передадим в него список объектов, полученный из БД; выводить будем по 10 записей на страницу.

* Получим нужную страницу пагинатора: ожидаем, что в запросе будет параметр `page=<номер_страницы>` — вот по этому значению и запросим нужную страницу.

* Вместо полного списка объектов передадим в шаблон страницу пагинатора.

* Подправим HTML-шаблон: добавим и настроим блок навигации по страницам.

Внесите изменения во view-функцию `birthday_list()`:

```py
# birthday/views.py
# Импортируем класс пагинатора.
from django.core.paginator import Paginator
...

def birthday_list(request):
    # Получаем список всех объектов с сортировкой по id.
    birthdays = Birthday.objects.order_by('id')
    # Создаём объект пагинатора с количеством 10 записей на страницу.
    paginator = Paginator(birthdays, 10)

    # Получаем из запроса значение параметра page.
    page_number = request.GET.get('page')
    # Получаем запрошенную страницу пагинатора. 
    # Если параметра page нет в запросе или его значение не приводится к числу,
    # вернётся первая страница.
    page_obj = paginator.get_page(page_number)
    # Вместо полного списка объектов передаём в контекст 
    # объект страницы пагинатора
    context = {'page_obj': page_obj}
    return render(request, 'birthday/birthday_list.html', context)
```

После изменения view-функции необходимо поправить и HTML-шаблон *birthday/birthday_list.htm*l: заменить название переменной `birthdays` на `page_obj` — ведь список объектов теперь передаётся в `page_obj`.

***
## Управление страницами пагинатора

Во view-функции в переменную page_obj передаётся объект страницы с запрошенным номером page_number:

```py
page_obj = paginator.get_page(page_number) 
```

В свойстве `page_obj.object_list` объект страницы хранит список объектов, выбранных для указанной страницы.

В этом объекте есть и несколько полезных методов:

* `page_obj.has_next()` — тип `bool`: есть ли страница со следующим номером;

* `page_obj.has_previous()` — тип `bool`: есть ли страница с предыдущим номером;

* `page_obj.has_other_pages()` — тип `bool`: есть ли другие страницы, кроме полученной;

* `page_obj.next_page_number()` — тип `int`: каков номер следующей страницы;

* `page_obj.previous_page_number()` — тип `int`: каков номер предыдущей страницы;

* `page_obj.start_index()` — тип: `int`: каков номер первого элемента на текущей странице. Отсчёт идёт от начала списка, начиная с 1. Например, если объекты распределяются по пять на одну страницу, а `page_obj` — это третья страница, то `page_obj.start_index()` вернёт 11;

* `page_obj.end_index()` — тип: `int`: каков номер последнего элемента на текущей странице. Отсчёт идёт от начала списка, начиная с 1. Например, если объекты распределяются по пять на одну страницу, а `page_obj` — это третья страница, то `page_obj.end_index()` вернёт 15.

Методы `start_index()` и `end_index()` бывают полезны, когда нужно нумеровать элементы, выведенные на страницу.

***
## Навигация по страницам

Помимо списка объектов переменная `page_obj` содержит объект `paginator`, в котором доступны все атрибуты и методы класса `Paginator`. К этому объекту можно обратиться, запросив его методы или атрибуты; это пригодится при работе с шаблоном:

* **page_obj.paginator.per_page** — число: количество записей на странице;

* **page_obj.paginator.num_pages** — число: количество страниц в пагинаторе;

* **page_obj.paginator.page_range** — итератор: хранит диапазон номеров всех страниц пагинатора — [1, 2, 3, 4, …].

Узнать о дополнительных возможностях класса `Paginator` можно [из документации](https://docs.djangoproject.com/en/3.2/ref/paginator/#django.core.paginator.Paginator).

Применим эту теорию к делу: напишем HTML-шаблон, генерирующий строку с навигацией по страницам пагинатора.

Создайте в директории *templates/includes* шаблон *paginator.html*. 

Все классы в шаблоне — из фреймворка Bootstrap, в нём подготовлены CSS-стили для оформления постраничной навигации. [В документации Bootstrap описана и стандартная структура вёрстки пагинатора](https://getbootstrap.com/docs/4.0/components/pagination/).

```html
<!-- templates/includes/paginator.html -->
{% if page_obj.has_other_pages %}
  <nav class="my-5">
    <ul class="pagination">
      <!-- Если существует предыдущая страница 
           (если мы не на первой странице) 
           рисуем кнопку "Первая страница"... -->
      {% if page_obj.has_previous %}
        <li class="page-item">
          <a class="page-link" href="?page=1">Первая</a>
        </li>
        <!-- ...и кнопку "Предыдущая" -->
        <li class="page-item">
          <a class="page-link" href="?page={{ page_obj.previous_page_number }}">
            Предыдущая
          </a>
        </li>
      {% endif %}
      <!-- Перебираем в цикле все номера страниц -->
      {% for i in page_obj.paginator.page_range %}
        <!-- Если номер страницы совпадает с i... -->
        {% if page_obj.number == i %}
          <!-- ..."подсвечиваем" кнопку: ставим класс "active"
               и делаем не ссылку, а span (просто текстовый блок) -->
          <li class="page-item active">
            <span class="page-link">{{ i }}</span>
          </li>
        {% else %}
          <!-- Остальные кнопки отрисовываем без подсветки, с ссылками -->
          <li class="page-item">
            <a class="page-link" href="?page={{ i }}">{{ i }}</a>
          </li>
        {% endif %}
      {% endfor %}

      {% if page_obj.has_next %}
        <!-- Если существует следующая страница 
           (если мы не на последней странице) 
           рисуем кнопку "Следующая"... -->
        <li class="page-item">
          <a class="page-link" href="?page={{ page_obj.next_page_number }}">
            Следующая
          </a>
        </li>
        <!-- ...и кнопку "Последняя" -->
        <li class="page-item">
          <a class="page-link" href="?page={{ page_obj.paginator.num_pages }}">
            Последняя
          </a>
        </li>
      {% endif %}
    </ul>
  </nav>
{% endif %} 
```

![alt text](https://pictures.s3.yandex.net/resources/184_1685550272.png)


Осталось подключить код пагинатора к HTML-шаблону со списком записей. Допишите в шаблон *birthday_list.html* строчку с тегом `{% include %}` и проверьте, что всё работает.

```html
<!-- birthday/birthday_list.html -->
{% extends "base.html" %}

{% block content %}
  {% for birthday in page_obj %}
    <div class="row">   

      <div class="col-2">  
        {% if birthday.image %}
          <img src="{{ birthday.image.url }}" height=100>
        {% endif %}
      </div>

      <div class="col-10">  
        <div>
          {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}
        </div>      
        <div>
          <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a> | <a href="{% url 'birthday:delete' birthday.id %}">Удалить запись</a>
        </div>
      </div>

      {% if not forloop.last %}
        <hr class="mt-3">
      {% endif %}
    </div>
  {% endfor %}

  <!-- Подключаем пагинатор -->
  {% include "includes/paginator.html" %}

{% endblock %} 
```

Сохраняйте в закладки [шпаргалку](https://code.s3.yandex.net/Python-dev/cheatsheets/034-django-paginator-shpora/034-django-paginator-shpora.html) к уроку. В ней — база о постраничном выводе информации и все методы страницы пагинатора.